
#### ## OBJETIVO: promedio de las mediciones de AERONET 
para un intervalo de tiempo determinado centrado en el paso del sat?lite,
para compararlo con el promedio de las recuperaciones de MODIS.

In [4]:
#Librerias
import pandas as pd
import numpy as np
from datetime import timedelta


In [48]:

def time_correlation(path_aeronet, path_modis, time_buffer):

    # --- AERONET ---
    data_aeronet = pd.read_csv(path_aeronet, na_values="NA")
    data_aeronet = data_aeronet.dropna()
    data_aeronet["date"] = (
        pd.to_datetime(
            data_aeronet["date"],
            format="%d/%m/%Y %H:%M"
        )
        .dt.tz_localize("GMT")
    )

    # --- MODIS / MAIAC ---
    data_sat = pd.read_csv(path_modis, na_values="NA")
    data_modis = data_sat.dropna(subset=["AOD"])
    data_modis["datetime"] = (
    pd.to_datetime(
        data_modis["dia"].astype(str) + " " + data_modis["hora"].astype(str),
        format="%d/%m/%Y %H:%M:%S"
    )
    .dt.tz_localize("GMT")
)

    

    AOD = []

    for i in range(len(data_modis)):

        if i % 50 == 0:
            print(i)

        row = data_modis.iloc[i]
        table_aeronet = data_aeronet[
    (data_aeronet["date"].dt.year == row["datetime"].year) &
    (data_aeronet["date"].dt.month == row["datetime"].month) &
    (data_aeronet["date"].dt.day == row["datetime"].day)
]

        

        if table_aeronet.empty:
            continue

        # === Ventana temporal ===
        
        diffs = abs(table_aeronet["date"] - row["datetime"])

        table_dif = table_aeronet[diffs < timedelta(minutes=time_buffer)]
        
        if table_dif.empty:
            continue

        # === Estadísticos AERONET ===
        mean_aer = table_dif.iloc[:, 3].mean()
        med_aer  = table_dif.iloc[:, 3].median()
        sd_aer   = table_dif.iloc[:, 3].std()
        dim_aer  = len(table_dif)
        #print(mean_aer)
        df = {
            "Date_MODIS": row.iloc[1],
            "AOD_550_modis": row.iloc[3],
            "satellite": row.iloc[6],
            "Date_AERONET": table_dif.iloc[0, 0].strftime("%Y-%m-%d"),
            "AOD_550_AER_mean": mean_aer,
            "AOD_550_AER_median": med_aer,
            "AOD_550_AER_sd": sd_aer,
            "AOD_550_AER_dim": dim_aer
        }

        AOD.append(df)
    #print(row)
    AOD = pd.DataFrame(AOD)
    AOD = AOD.dropna()

    return AOD

buffer_time = 60
buffer_spatial = "1km"
city = "GT"
num_site = "2"
region = "USA"
formato_fecha = "%d/%m/%Y %H:%M"

# data_maiac = f"D:/Josefina/paper_git/paper_maiac/datasets/V03/maiac/{region}_C61/{city}/prueba_{buffer_spatial}_{city}_C61_tot_V03.csv"
# data_aeronet = f"D:/Josefina/paper_git/paper_maiac/datasets/V03/aeronet/datasets_interp_s_L02/{region}/{num_site}_{city}_2015-2024_interp-s_V03_L2.csv"

data_maiac = "D:/Josefina/Proyectos/MAIAC/V02/Code/02_Datasets/MODIS/BA-3KM-MODIS.csv"
data_aeronet = "D:/Josefina/Proyectos/MAIAC/V02/Code/02_Datasets/AERONET/3_BA_2015-2024_AERONET_L2.csv"
combinate = time_correlation(
    path_aeronet=data_aeronet,
    path_modis=data_maiac,
    time_buffer=buffer_time,
    #formato_fecha=formato_fecha
)

# combinate.to_csv(
#     f"D:/Josefina/paper_git/paper_maiac/datasets/V03/processed/merge_AER-MAIAC/{region}_C61/tot/{buffer_spatial}/{num_site}_{city}-{buffer_spatial}-MAIAC-{buffer_time}-AER_C61.csv",
#     index=False
# )




C:\Users\Josefina Urquiza\AppData\Local\Temp\ipykernel_17000\1454905757.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_modis["datetime"] = (


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050


In [ ]:
#Lo visualizamos
combinate

,Date_MODIS,AOD_550_modis,satellite,Date_AERONET,AOD_550_AER_mean,AOD_550_AER_median,AOD_550_AER_sd,AOD_550_AER_dim
0,2/1/2015,0.3570,MOD,2015-01-02,0.026027,0.025059,0.004054,9
1,29/1/2015,0.1160,MOD,2015-01-29,0.060724,0.058999,0.007071,8
2,6/2/2015,0.5560,MOD,2015-02-06,0.064835,0.066920,0.007662,9
3,20/2/2015,0.3770,MOD,2015-02-20,0.239509,0.270695,0.065847,8
4,22/2/2015,0.5395,MOD,2015-02-22,0.113714,0.111769,0.023510,8
...,...,...,...,...,...,...,...,...
710,14/11/2024,0.2350,MOD,2024-11-14,0.064646,0.065913,0.006243,22
711,19/11/2024,0.4020,MOD,2024-11-19,0.065743,0.062577,0.010068,5
712,20/11/2024,0.0475,MOD,2024-11-20,0.055690,0.055112,0.002301,21
713,2/12/2024,0.0420,MOD,2024-12-02,0.050977,0.050998,0.004864,19


In [50]:
###############################################################################
# PROMEDIOS DIARIOS
def promedios(combinate):
    combinate = combinate.copy()

    # Convertir Date_MODIS a datetime (GMT)
    combinate["date"] = pd.to_datetime(
        combinate["Date_MODIS"],
        format="%Y-%m-%d %H:%M",
        utc=True
    )

    # Agrupar por día (si querés SOLO día, sin hora)
    combinate["day"] = combinate["date"].dt.date

    rbind_combinate = []

    for day, group in combinate.groupby("day"):
        df = {
            "date": day,
            "AOD_550_modis": group["AOD_550_modis"].mean(skipna=True),
            "AOD_550_AER_mean": group["AOD_550_AER_mean"].mean(skipna=True)
        }
        rbind_combinate.append(df)

    return pd.DataFrame(rbind_combinate)
